In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
with open('df_dump.dat', 'br') as file:
    a = pickle.load(file)

a['warehouse'].rename(columns={'offer_id': 'doc_id', 'report_dt': 'report_dttm', 'inflow': 'income'}, inplace=True)
a['cancels'].rename(columns={'offer_id': 'doc_id', 'report_dt': 'report_dttm'}, inplace=True)
a['deficit'].rename(columns={'offer_id': 'doc_id', 'report_dt': 'report_dttm'}, inplace=True)

In [ ]:
a.keys()

In [ ]:
a['full_warehouse']['consumption'] = -a['full_warehouse']['consumption'].astype('float64')
a['full_warehouse']['income'] = a['full_warehouse']['income'].astype('float64')
a['full_warehouse']

In [ ]:
pd.concat([a['cancels'].rename(columns={'count': 'cancels'}),
           a['deficit'].rename(columns={'count': 'deficit'})])

In [ ]:
a['full_warehouse'] \
    .groupby(by='category', as_index=False) \
    .agg({'report_dttm': 'count'}) \
    .rename({'report_dttm' : 'operation_count'}, axis=1) \
    .sort_values(by='operation_count', ascending=False)

In [ ]:
category = a['full_warehouse'].category.unique()[3]
category

In [ ]:
nomenclature = a['full_warehouse'].nomenclature.unique().tolist()
nomenclature

In [ ]:
journal = []

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[0]')[['income', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'income': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[1]')[['consumption', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'consumption': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[2]')[['consumption', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'consumption': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[3]')[['consumption', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'consumption': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[4]')[['income', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'income': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[5]')[['consumption', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'consumption': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[6]')[['income', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'income': 'value'}, axis=1) \
                   .dropna() \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[6]')[['consumption', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'consumption': 'value'}, axis=1) \
                   .dropna() \
                   .to_dict('r')

In [ ]:
journal += a['full_warehouse'].query('nomenclature == @nomenclature[7]')[['income', 'category', 'doc_id', 'report_dttm', 'nomenclature']] \
                   .rename({'nomenclature': 'metric', 'income': 'value'}, axis=1) \
                   .to_dict('r')

In [ ]:
a['cancels']['count'] = a['cancels']['count'].astype('float64')
journal += a['cancels'].assign(metric='Отказ клиента').rename({'count': 'value'}, axis=1).to_dict('r')

In [ ]:
a['deficit']['count'] = a['deficit']['count'].astype('float64')
journal += a['deficit'].assign(metric='Дефицит товара').rename({'count': 'value'}, axis=1).to_dict('r')

In [ ]:
pd.DataFrame(journal).columns.tolist()

In [ ]:
metric_columns = ['category', 'doc_id', 'metric']
t = pd.DataFrame(journal).set_index('report_dttm') \
                     .groupby([pd.Grouper(freq='1M'), 'category', 'metric']) \
                     .sum() \
                     .unstack() \
                     .reset_index() 
t

In [ ]:
for key, group in t.groupby('category'):
    group.drop('category', axis=1).set_index('report_dttm').plot(kind='bar', stacked=True, figsize=(25, 10))
    plt.tight_layout()